# Assignment 1: Search
# Part 2:

In [32]:
from search_1_2 import *
from itertools import combinations, permutations
import numpy as np
import time
    
fname = 'src/miniSearch.txt'
# sname_dfs = fname[:-4] + '_sol_dfs' + fname[-4:]
sname_bfs_1 = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_bfs_1' + fname[-4:]
sname_bfs_2 = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_bfs_2' + fname[-4:]
sname_bfs_3 = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_bfs_3' + fname[-4:]
sname_gbfs = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_gbfs' + fname[-4:]
sname_astar_1 = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_astar_1' + fname[-4:]
sname_astar_2 = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_astar_2' + fname[-4:]


# Breadth First Search (1)


In [33]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
# print(index_dict)
timer_s = time.clock()

curr_node = start
while index_dict:
    dist = []
    for goal in index_dict:
        manhattan = get_manhattan(curr_node, goal)
        dist.append((manhattan, goal))
    if len(dist) == 1:
        print('DONE')
        break
    dist.sort()
#     print(dist)
    
    if len(dist) > 2:
        real_dist = []
        for i in range(1, len(dist) - 1):
            goal = dist[i][1]
            path, expanded = find_path_bfs(maze_map, start, goal)
            total_expanded += expanded
            bfs = len(path)
            real_dist.append((bfs, goal))
            if bfs > dist[i + 1][0]:
                break

        real_dist.sort()
#         print(real_dist)
        next_node = real_dist[0][1]
    else:
        next_node = dist[1][1]

    path, expanded = find_path_bfs(maze_map, curr_node, next_node)
    total_path += path
    
    print(curr_node, '--->>', next_node)
    curr_index = index_dict[curr_node]
    del index_dict[curr_node]
    del goal_dict[curr_index]
    curr_node = next_node

timer_t = time.clock()
print('bfs takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_bfs = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_bfs_1, maze_sol_bfs)


(4, 4) --->> (5, 4)
(5, 4) --->> (4, 2)
(4, 2) --->> (2, 3)
(2, 3) --->> (1, 1)
(1, 1) --->> (3, 6)
(3, 6) --->> (3, 8)
(3, 8) --->> (1, 8)
(1, 8) --->> (5, 7)
(5, 7) --->> (6, 8)
(6, 8) --->> (7, 1)
DONE
bfs takes 0.006069999999997577 seconds, costs 44 steps and expands 370 cells.


# Breadth First Search (2)


In [34]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
# print(index_dict)
timer_s = time.clock()

length = len(goal_dict.keys())

t = Tree(length)
dist_dict = {}
for goal_i, goal_j in combinations(index_dict.keys(), r = 2):
    path, expanded = find_path_bfs(maze_map, goal_i, goal_j)
    total_expanded += expanded
    dist = len(path)
    dist_dict[(goal_i, goal_j)] = dist
    dist_dict[(goal_j, goal_i)] = dist
    i = index_dict[goal_i]
    j = index_dict[goal_j]
    t.addEdge(i, j, dist)
    # print(i, j)
mst = t.KruskalMST()
# print(mst)

curr_index = index_dict[start]
next_index = curr_index

while goal_dict.keys():
    
    length = len(goal_dict.keys())
    if length == 1:
        print('DONE')
        break
    
    edges = []
    for edge in mst:
        # print(curr_index)
        # print(edge)
        if edge[0] == curr_index or edge[1] == curr_index:
            edges.append(edge)
    for edge in edges:
        mst.remove(edge)
    
    if edges == []:
        dist_list = []
        curr_node = goal_dict[curr_index]
        for goal in index_dict.keys():
            if (curr_node, goal) in dist_dict.keys():
                dist_list.append((dist_dict[(curr_node, goal)], goal))
            else:
                path, expanded = find_path_bfs(maze_map, curr_node, goal)
                total_expanded += expanded
                dist = len(path)
                dist_list.append((dist, goal))
        dist_list.sort()
        if len(dist_list) == 1:
            print(index_dict)
            print(dist_list)
        next_node = dist_list[1][1]
    else:
        sorted(edges, key=lambda edge: edge[2])
        if edges[0][1] == curr_index:
            next_index = edges[0][0]
        else: 
            next_index = edges[0][1]
        curr_node = goal_dict[curr_index]
        next_node = goal_dict[next_index]
    
    print(curr_node, '--->>', next_node)
    path, expanded = find_path_astar(maze_map, curr_node, next_node)
    
    total_path += path
    
    del goal_dict[curr_index]
    del index_dict[curr_node]
    curr_node = next_node
    curr_index = index_dict[curr_node]

timer_t = time.clock()
print('bfs takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_bfs = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_bfs_2, maze_sol_bfs)


(4, 4) --->> (5, 4)
(5, 4) --->> (4, 2)
(4, 2) --->> (7, 1)
(7, 1) --->> (1, 1)
(1, 1) --->> (2, 3)
(2, 3) --->> (3, 6)
(3, 6) --->> (5, 7)
(5, 7) --->> (6, 8)
(6, 8) --->> (3, 8)
(3, 8) --->> (1, 8)
DONE
bfs takes 0.008589000000000624 seconds, costs 31 steps and expands 1170 cells.


# Breadth First Search (3)


In [35]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
# print(index_dict)
timer_s = time.clock()
length = len(goal_dict)
dist_dict = {}
path_dict = {}

for goal_i, goal_j in combinations(index_dict.keys(), r = 2):
    path, expanded = find_path_bfs(maze_map, goal_i, goal_j)
    total_expanded += expanded
    dist = len(path)
    dist_dict[(goal_i, goal_j)] = dist
    dist_dict[(goal_j, goal_i)] = dist
    path_dict[(goal_i, goal_j)] = path

goals = list(index_dict.keys())
goals.remove(start)
# print(goals)

min_dist = 9999
min_path = []

for goal_order in permutations(goals):
    temp_dist = 0
    temp_path = [start]
    curr_node = start
    next_node = ()
    for goal in goal_order:
        next_node = goal
        dist = dist_dict[(curr_node, next_node)]
        temp_dist += dist
        temp_path.append(next_node)
        curr_node = next_node
    if temp_dist < min_dist:
#         print(temp_dist)
        min_dist = temp_dist
        min_path = temp_path
# print(min_path)

curr_node = start
next_node = ()
for i in range(1, len(min_path)):
    next_node = min_path[i]
    print(curr_node, '--->>', next_node)
    if (curr_node, next_node) in path_dict:
        path = path_dict[(curr_node, next_node)]
    else:
        path, _ = find_path_bfs(maze_map, curr_node, next_node)
#     print(path)
    total_path += path
    curr_node = next_node
    
timer_t = time.clock()
# print(total_path)
print('bfs takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_bfs = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_bfs_3, maze_sol_bfs)


(4, 4) --->> (5, 4)
(5, 4) --->> (4, 2)
(4, 2) --->> (7, 1)
(7, 1) --->> (1, 1)
(1, 1) --->> (2, 3)
(2, 3) --->> (3, 6)
(3, 6) --->> (5, 7)
(5, 7) --->> (6, 8)
(6, 8) --->> (3, 8)
(3, 8) --->> (1, 8)
bfs takes 12.418965999999998 seconds, costs 31 steps and expands 1170 cells.


# Greedy Best First Search (1)

In [36]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
# print(index_dict)
timer_s = time.clock()

length = len(goal_dict.keys())

t = Tree(length)
dist_dict = {}
for goal_i, goal_j in combinations(index_dict.keys(), r = 2):
    path, expanded = find_path_gbfs(maze_map, goal_i, goal_j)
    total_expanded += expanded
    dist = len(path)
    dist_dict[(goal_i, goal_j)] = dist
    dist_dict[(goal_j, goal_i)] = dist
    i = index_dict[goal_i]
    j = index_dict[goal_j]
    t.addEdge(i, j, dist)
    # print(i, j)
mst = t.KruskalMST()
# print(mst)

curr_index = index_dict[start]
next_index = curr_index

while goal_dict.keys():
    length = len(goal_dict.keys())
    if length == 1:
        print('DONE')
        break
    
    edges = []
    for edge in mst:
        # print(curr_index)
        # print(edge)
        if edge[0] == curr_index or edge[1] == curr_index:
            edges.append(edge)
    for edge in edges:
        mst.remove(edge)
    
    if edges == []:
        dist_list = []
        curr_node = goal_dict[curr_index]
        for goal in index_dict.keys():
            if (curr_node, goal) in dist_dict.keys():
                dist_list.append((dist_dict[(curr_node, goal)], goal))
            else:
                path, expanded = find_path_gbfs(maze_map, curr_node, goal)
                total_expanded += expanded
                dist = len(path)
                dist_list.append((dist, goal))
        dist_list.sort()
        if len(dist_list) == 1:
            pass
#             print(index_dict)
#             print(dist_list)
        next_node = dist_list[1][1]
    else:
        sorted(edges, key=lambda edge: edge[2])
        if edges[0][1] == curr_index:
            next_index = edges[0][0]
        else: 
            next_index = edges[0][1]
        curr_node = goal_dict[curr_index]
        next_node = goal_dict[next_index]
    
    print(curr_node, '--->>', next_node)
    path, expanded = find_path_astar(maze_map, curr_node, next_node)
    
    total_path += path
    
    del goal_dict[curr_index]
    del index_dict[curr_node]
    curr_node = next_node
    curr_index = index_dict[curr_node]

timer_t = time.clock()
print('gbfs takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_gbfs = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_gbfs, maze_sol_gbfs)


(4, 4) --->> (5, 4)
(5, 4) --->> (4, 2)
(4, 2) --->> (7, 1)
(7, 1) --->> (1, 1)
(1, 1) --->> (2, 3)
(2, 3) --->> (3, 6)
(3, 6) --->> (5, 7)
(5, 7) --->> (6, 8)
(6, 8) --->> (3, 8)
(3, 8) --->> (1, 8)
DONE
gbfs takes 0.007865000000002453 seconds, costs 31 steps and expands 354 cells.


# A* Search (1)

In [37]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
# print(index_dict)
timer_s = time.clock()

curr_node = start
while index_dict:
    dist = []
    for goal in index_dict:
        manhattan = get_manhattan(curr_node, goal)
        dist.append((manhattan, goal))
    if len(dist) == 1:
        print('DONE')
        break
    dist.sort()
#     print(dist)
    
    if len(dist) > 2:
        real_dist = []
        for i in range(1, len(dist) - 1):
            goal = dist[i][1]
            path, expanded = find_path_astar(maze_map, start, goal)
            total_expanded += expanded
            bfs = len(path)
            real_dist.append((bfs, goal))
            if bfs > dist[i + 1][0]:
                break

        real_dist.sort()
#         print(real_dist)
        next_node = real_dist[0][1]
    else:
        next_node = dist[1][1]

    path, expanded = find_path_astar(maze_map, curr_node, next_node)
    total_path += path
    
    print(curr_node, '--->>', next_node)
    curr_index = index_dict[curr_node]
    del index_dict[curr_node]
    del goal_dict[curr_index]
    curr_node = next_node

timer_t = time.clock()
print('astar takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_astar = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_astar_1, maze_sol_astar)


(4, 4) --->> (5, 4)
(5, 4) --->> (4, 2)
(4, 2) --->> (2, 3)
(2, 3) --->> (1, 1)
(1, 1) --->> (3, 6)
(3, 6) --->> (3, 8)
(3, 8) --->> (1, 8)
(1, 8) --->> (5, 7)
(5, 7) --->> (6, 8)
(6, 8) --->> (7, 1)
DONE
astar takes 0.0053549999999944475 seconds, costs 44 steps and expands 91 cells.


# A* Search (2)

In [38]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
# print(index_dict)
timer_s = time.clock()

length = len(goal_dict.keys())

t = Tree(length)
dist_dict = {}
for goal_i, goal_j in combinations(index_dict.keys(), r = 2):
    path, expanded = find_path_astar(maze_map, goal_i, goal_j)
    total_expanded += expanded
    dist = len(path)
    dist_dict[(goal_i, goal_j)] = dist
    dist_dict[(goal_j, goal_i)] = dist
    i = index_dict[goal_i]
    j = index_dict[goal_j]
    t.addEdge(i, j, dist)
    # print(i, j)
mst = t.KruskalMST()
print(mst)

curr_index = index_dict[start]
next_index = curr_index

while goal_dict.keys():
    length = len(goal_dict.keys())
    if length == 1:
        print('DONE')
        break
    
    edges = []
    for edge in mst:
        # print(curr_index)
        # print(edge)
        if edge[0] == curr_index or edge[1] == curr_index:
            edges.append(edge)
    for edge in edges:
        mst.remove(edge)
    
    if edges == []:
        dist_list = []
        curr_node = goal_dict[curr_index]
        for goal in index_dict.keys():
            if (curr_node, goal) in dist_dict.keys():
                dist_list.append((dist_dict[(curr_node, goal)], goal))
            else:
                path, expanded = find_path_astar(maze_map, curr_node, goal)
                total_expanded += expanded
                dist = len(path)
                dist_list.append((dist, goal))
        dist_list.sort()
        if len(dist_list) == 1:
            print(index_dict)
            print(dist_list)
        next_node = dist_list[1][1]
    else:
        sorted(edges, key=lambda edge: edge[2])
        if edges[0][1] == curr_index:
            next_index = edges[0][0]
        else: 
            next_index = edges[0][1]
        curr_node = goal_dict[curr_index]
        next_node = goal_dict[next_index]
    
    print(curr_node, '--->>', next_node)
    path, expanded = find_path_astar(maze_map, curr_node, next_node)
    
    total_path += path
    
    del goal_dict[curr_index]
    del index_dict[curr_node]
    curr_node = next_node
    curr_index = index_dict[curr_node]

timer_t = time.clock()
print('astar takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_astar = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_astar_2, maze_sol_astar)


[[6, 7, 1], [1, 4, 2], [8, 9, 2], [0, 2, 3], [2, 6, 3], [3, 6, 3], [3, 8, 3], [4, 8, 3], [5, 7, 3], [5, 10, 4]]
(4, 4) --->> (5, 4)
(5, 4) --->> (4, 2)
(4, 2) --->> (7, 1)
(7, 1) --->> (1, 1)
(1, 1) --->> (2, 3)
(2, 3) --->> (3, 6)
(3, 6) --->> (5, 7)
(5, 7) --->> (6, 8)
(6, 8) --->> (3, 8)
(3, 8) --->> (1, 8)
DONE
astar takes 0.009374999999998579 seconds, costs 31 steps and expands 411 cells.
